In [1]:
import os
import subprocess
import trimesh
import tqdm
import numpy as np
import h5py
from utils import MeshLoader, ProcessKillingExecutor, process_mesh

In [2]:
grasps_dir = "/root/share/pkgs/SceneCollisionNet/datasets/grasps"
mesh_dir = "/root/share/pkgs/SceneCollisionNet/datasets/ShapeNetSem/models-COLLADA/COLLADA_mini"
out_dir = "/root/share/pkgs/SceneCollisionNet/datasets/shapenet"

In [3]:
mesh_loader = MeshLoader(mesh_dir)

In [4]:
out_ds_file = os.path.join(out_dir, "object_info.hdf5")

In [5]:
inputs = []
for f in os.listdir(grasps_dir):
    mc, mk, ms = os.path.splitext(f)[0].split("_") # category, key, scale
    try:
        in_path = mesh_loader.get_path(mk)
        print(in_path)
    except ValueError:
        continue
    out_path = os.path.join(
        out_dir,
        mc,
        os.path.splitext(os.path.basename(in_path))[0] + ".obj",
    )
    inputs.append(
        (in_path, out_path, float(ms), os.path.join(grasps_dir, f))
    )

/root/share/pkgs/SceneCollisionNet/datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a4ec387ea6820345778775dfd5ca46a.dae
/root/share/pkgs/SceneCollisionNet/datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a6a67905880e4911a4d5e0a785b0e03.dae
/root/share/pkgs/SceneCollisionNet/datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a5e21a5633e1134ebd90840d80c9784.dae
/root/share/pkgs/SceneCollisionNet/datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a3efcaaf8db9957a010c31b9816f48b.dae
/root/share/pkgs/SceneCollisionNet/datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a4daa4904bb4a0949684e7f0bb99f9c.dae


In [6]:
def process_mesh_timeout(*args, **kwargs):
    pass

In [15]:
executor = ProcessKillingExecutor(max_workers=8)
generator = executor.map(
    process_mesh,
    inputs,
    timeout=120,
    callback_timeout=process_mesh_timeout,
)

concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts


In [18]:
with h5py.File(out_ds_file, "a") as f:
    if "meshes" not in f:
        f.create_group("meshes")

    categories = {}
    if "categories" not in f:
        f.create_group("categories")

    for mesh_info in generator:
        if mesh_info is not None:
            mk, minfo = mesh_info # mesh name, mesh info
            if mk not in f["meshes"]:
                f["meshes"].create_group(mk)
            for key in minfo:
                if key in f["meshes"][mk]:
                    del f["meshes"][mk][key]
                f["meshes"][mk][key] = minfo[key] # path, scale, category, stps, probs, grasps

            if minfo["category"] not in categories:
                categories[minfo["category"]] = [mk.encode('utf8')] # string 저장 시 utf8로 인코딩하지 않으면 에러 발생
            elif mk not in categories[minfo["category"]]:
                categories[minfo["category"]].append(mk.encode('utf8')) # string 저장 시 utf8로 인코딩하지 않으면 에러 발생
            
            category = minfo["category"]
            if category in f["categories"]:
                del f["categories"][category]
            f["categories"][category] = categories[category]

In [19]:
with h5py.File(out_ds_file, "a") as f:
    meshes = list(f["meshes"].keys())
    print(meshes)
    print(f["meshes"][meshes[0]]['path'][()])

['1a3efcaaf8db9957a010c31b9816f48b_0.011789003201190232', '1a4daa4904bb4a0949684e7f0bb99f9c_0.00040353195213810913', '1a4ec387ea6820345778775dfd5ca46a_0.004405836004468139', '1a5e21a5633e1134ebd90840d80c9784_0.02338623474048206', '1a6a67905880e4911a4d5e0a785b0e03_0.0011058383813692342']
Laptop/1a3efcaaf8db9957a010c31b9816f48b_0.011789003201190232.obj
